In [1]:
import yaml
import epo_ops
import pandas as pd
import xml.etree.ElementTree as ET
import re
import numpy as np
import json
import time
import requests
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
import html
import random

In [2]:
# Load patstat identifiers 
Sustainable_AI = pd.read_csv("Sustainable_AI.csv", delimiter = ";")

## Step 1: Scrape relevant patents from EPO OPS API

In [3]:
# Open config file with EPO OPS credentials
with open('config_file.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [38]:
# Function to get text from element or return None if element is None
def get_text(element):
    return element.text if element is not None else ""

# Function to parse bibliographic data
def parse_biblio(bibliographic_data, Patent_dict): 
    # Parse the XML
    root = ET.fromstring(bibliographic_data.text)

    # Define namespaces
    namespaces = {
    'ops': 'http://ops.epo.org',
    '': 'http://www.epo.org/exchange'
    }

    
    # Extract basic bibliographic data
    biblio_data = root.find('.//{http://www.epo.org/exchange}bibliographic-data')
    country = biblio_data.find('.//{http://www.epo.org/exchange}publication-reference/{http://www.epo.org/exchange}document-id/{http://www.epo.org/exchange}country').text
    doc_number = biblio_data.find('.//{http://www.epo.org/exchange}publication-reference/{http://www.epo.org/exchange}document-id/{http://www.epo.org/exchange}doc-number').text
    kind = biblio_data.find('.//{http://www.epo.org/exchange}publication-reference/{http://www.epo.org/exchange}document-id/{http://www.epo.org/exchange}kind').text
    date = biblio_data.find('.//{http://www.epo.org/exchange}publication-reference/{http://www.epo.org/exchange}document-id/{http://www.epo.org/exchange}date').text

    publication = country + doc_number + kind
    Patent_dict[publication] = {'country': country, 'number': doc_number, 'kind': kind, 'date': date}

    # Title
    Patent_dict[publication]['title_de'] = None
    Patent_dict[publication]['title_en'] = None
    Patent_dict[publication]['title_fr'] = None
    titles = biblio_data.findall('.//{http://www.epo.org/exchange}invention-title')
    for title in titles:
        lang = title.attrib['lang']
        Patent_dict[publication]['title_'+lang] = get_text(title)

    # Abstracts
    Patent_dict[publication]['abstract_de'] = None
    Patent_dict[publication]['abstract_en'] = None
    Patent_dict[publication]['abstract_fr'] = None
    abstracts = root.findall('.//{http://www.epo.org/exchange}abstract')
    for abstract in abstracts:
        lang = abstract.attrib['lang']
        abstract_texts = [get_text(p) for p in abstract.findall('.//{http://www.epo.org/exchange}p')]
        Patent_dict[publication]['abstract_'+lang] = ' '.join(abstract_texts)
    
    # Extracting applicants
    applicants = biblio_data.findall('.//{http://www.epo.org/exchange}applicants/{http://www.epo.org/exchange}applicant')
    applicant_names = []
    applicant_countries = []
    for applicant in applicants:
        data_format = applicant.attrib.get('data-format')
        if data_format == 'epodoc':
                # Extract the country code from the epodoc format
                name_element = applicant.find('.//{http://www.epo.org/exchange}name')
                if name_element is not None and '[' in name_element.text:
                    country_code = name_element.text.split('[')[-1].split(']')[0]
                    applicant_countries.append(country_code)
        elif data_format == 'original':
                # Extract the name from the original format
                name_element = applicant.find('.//{http://www.epo.org/exchange}name')
                if name_element is not None:
                    applicant_names.append(name_element.text)

    Patent_dict[publication]['applicant_names'] = applicant_names
    Patent_dict[publication]['applicant_countries'] = applicant_countries

    # Extracting inventors
    inventors = biblio_data.findall('.//{http://www.epo.org/exchange}inventors/{http://www.epo.org/exchange}inventor')
    inventor_names = []
    inventor_countries = []
    for inventor in inventors:
            data_format = inventor.attrib.get('data-format')
            if data_format == 'epodoc':
            # Extract the country code from the epodoc format
                name_element = inventor.find('.//{http://www.epo.org/exchange}name')
                if name_element is not None and '[' in name_element.text:
                    country_code = name_element.text.split('[')[-1].split(']')[0]
                    inventor_countries.append(country_code)
            elif data_format == 'original':
                # Extract the name from the original format
                name_element = inventor.find('.//{http://www.epo.org/exchange}name')
                if name_element is not None:
                    inventor_names.append(name_element.text)
    
    Patent_dict[publication]['inventor_names'] = inventor_names
    Patent_dict[publication]['inventor_countries'] = inventor_countries

    # Extracting IPC classes
    ipc_codes = []
    for ipc in root.findall('.//{http://www.epo.org/exchange}classification-ipc/{http://www.epo.org/exchange}text'):
        ipc_codes.append(get_text(ipc).strip())

    for ipcr in root.findall('.//{http://www.epo.org/exchange}classification-ipcr/{http://www.epo.org/exchange}text'):
        ipc_codes.append(get_text(ipcr).strip())

    Patent_dict[publication]['ipc_codes'] = ipc_codes

    # Extract CPC classes
    cpc_codes = []
    for patent_classification in root.findall('.//{http://www.epo.org/exchange}patent-classifications/{http://www.epo.org/exchange}patent-classification'):
        scheme = patent_classification.find('.//{http://www.epo.org/exchange}classification-scheme')
        if scheme is not None and scheme.attrib.get('scheme') == 'CPCI':
            section = get_text(patent_classification.find('.//{http://www.epo.org/exchange}section'))
            classification_class = get_text(patent_classification.find('.//{http://www.epo.org/exchange}class'))
            subclass = get_text(patent_classification.find('.//{http://www.epo.org/exchange}subclass'))
            main_group = get_text(patent_classification.find('.//{http://www.epo.org/exchange}main-group'))
            subgroup = get_text(patent_classification.find('.//{http://www.epo.org/exchange}subgroup'))
            
            cpc_code = f"{section}{classification_class}{subclass} {main_group}/{subgroup}"
            cpc_codes.append(cpc_code)
    
    Patent_dict[publication]['cpc_codes'] = cpc_codes

    # Family ID
    Patent_dict[publication]['family_id'] = root.find('.//{http://www.epo.org/exchange}exchange-document').attrib['family-id']
    
    # Application reference
    application_ref = biblio_data.find('.//{http://www.epo.org/exchange}application-reference')
    Patent_dict[publication]['application_number'] = get_text(application_ref.find('.//{http://www.epo.org/exchange}doc-number'))

    # URL (if available)
    Patent_dict[publication]['url'] = root.find('.//{http://www.epo.org/exchange}exchange-document').attrib.get('href')
    
    # instantiate additional keys which will be overwritten later on 
    Patent_dict[publication]['claims_de'] = None
    Patent_dict[publication]['claims_en'] = None
    Patent_dict[publication]['claims_fr'] = None
    Patent_dict[publication]['description_de'] = None
    Patent_dict[publication]['description_en'] = None
    Patent_dict[publication]['description_fr'] = None


    return Patent_dict


In [39]:
# Function to parse claims
def parse_claims(claims, Patent_dict):
    # Parse the XML
    root = ET.fromstring(claims.text)

    # Register namespaces to handle prefixed tags correctly
    namespaces = {
    'ops': 'http://ops.epo.org',
    'ftxt': 'http://www.epo.org/fulltext',
    '': 'http://www.epo.org/exchange'  # default namespace
    }

    # Extract publication details
    pub_ref = root.find('.//{http://www.epo.org/fulltext}fulltext-document/{http://www.epo.org/fulltext}bibliographic-data/{http://www.epo.org/fulltext}publication-reference/{http://www.epo.org/fulltext}document-id')
    country = pub_ref.find('{http://www.epo.org/fulltext}country').text
    doc_number = pub_ref.find('{http://www.epo.org/fulltext}doc-number').text
    kind = pub_ref.find('{http://www.epo.org/fulltext}kind').text

    # Add publication details to dictionary
    publication = country + doc_number + kind

    # Extract claim texts
    for claims_element in root.findall('.//{http://www.epo.org/fulltext}fulltext-document/{http://www.epo.org/fulltext}claims', namespaces):
        lang = claims_element.attrib.get('lang').lower()
        Patent_dict[publication]['claims_' + lang] = [claim_text_element.text for claim_text_element in claims_element.findall('{http://www.epo.org/fulltext}claim/{http://www.epo.org/fulltext}claim-text')]
    
    return Patent_dict

In [40]:
# Function to parse description 
def parse_description(description, Patent_dict):
    root = ET.fromstring(description.text)

    # Register namespaces to handle prefixed tags correctly
    namespaces = {
    'ops': 'http://ops.epo.org',
    'ftxt': 'http://www.epo.org/fulltext',
    '': 'http://www.epo.org/exchange'  # default namespace
    }   

    # Extract publication details
    pub_ref = root.find('.//{http://www.epo.org/fulltext}fulltext-document/{http://www.epo.org/fulltext}bibliographic-data/{http://www.epo.org/fulltext}publication-reference/{http://www.epo.org/fulltext}document-id')
    # Check if pub_ref was found
    country = pub_ref.find('{http://www.epo.org/fulltext}country').text
    doc_number = pub_ref.find('{http://www.epo.org/fulltext}doc-number').text
    kind = pub_ref.find('{http://www.epo.org/fulltext}kind').text

    publication = country + doc_number + kind
    
    # List of all description text
    all_description_text = []
    for description in root.findall('.//{http://www.epo.org/fulltext}fulltext-document/{http://www.epo.org/fulltext}description', namespaces):
        lang = description.attrib.get('lang').lower()
        # Iterate over all text elements under description
        for text_elem in description.findall('.//{http://www.epo.org/fulltext}p', namespaces):
            # Check if the text element has child elements
            if text_elem.findall('.//*'):
                # This might be a heading or has formatting, so iterate over its children
                text = ""
                for child in text_elem:
                    # Append text of child elements
                    if child.text:
                        text += child.text.strip() + " "
                # Append to all_description_text
                all_description_text.append(text.strip())
            else:
                # This is normal text
                # Remove numbers and append text to all_description_text
                text = re.sub(r'\[\d+\]', '', text_elem.text.strip())
                all_description_text.append(text.strip())

        Patent_dict[publication]['description_'+lang] = all_description_text

    return Patent_dict

In [34]:
Missing_pats = []
Patent_dictionary = {}
client = epo_ops.Client(key=config['KEY'], secret=config['SECRET'])
counter = 0
for row_full in Sustainable_AI.iterrows():
    row = row_full[1]
    print(str(row['PUBLN_NR']), row['publn_auth'], row['PUBLN_KIND'])
    counter += 1
    if counter % 500 == 0:
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        filename = f'Patent_dictionary_30000_{timestamp}.json'
        with open(filename, 'w') as json_file:
            json.dump(Patent_dictionary, json_file, indent=4)
        print(f"Saved intermediate file: {filename}, Counter: {counter}")
    try:
      bib_data = client.published_data(  # Retrieve bibliography data
        reference_type = 'publication',  # publication, application, priority
        input = epo_ops.models.Docdb(str(row['PUBLN_NR']), row['publn_auth'], row['PUBLN_KIND']),  
        endpoint = 'biblio', # optional, defaults to biblio in case of published_data
        constituents = None  # optional, list of constituents
        )
      Patent_dictionary = parse_biblio(bib_data, Patent_dictionary)
    except: 
        Missing_pats.append(str(row['PUBLN_NR']) + row['publn_auth'] + row['PUBLN_KIND'])
    try:
      claims_data = client.published_data(
        reference_type = 'publication',  # publication, application, priority
        input = epo_ops.models.Docdb(str(row['PUBLN_NR']), row['publn_auth'], row['PUBLN_KIND']),  
        endpoint = 'claims', # optional, defaults to biblio in case of published_data
        constituents = None  # optional, list of constituents
        )
      Patent_dictionary = parse_claims(claims_data, Patent_dictionary)
    except Exception as e:
      print(e)
      continue
    try:
      description_data = client.published_data(
        reference_type = 'publication',  # publication, application, priority
        input = epo_ops.models.Docdb(str(row['PUBLN_NR']), row['publn_auth'], row['PUBLN_KIND']),  
        endpoint = 'description', # optional, defaults to biblio in case of published_data
        constituents = None  # optional, list of constituents
        )
      Patent_dictionary = parse_description(description_data, Patent_dictionary)

    except Exception as e:
      print(e)
      continue


    
with open('Patent_dictionary.json', 'w') as json_file:
    json.dump(Patent_dictionary, json_file, indent=4)


with open('Missing_patents.txt', 'w') as file:
    for item in Missing_pats:
        file.write(f"{item}\n")


2296925 EP B1
2297277 EP A1
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297460 EP A2
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297482 EP A1
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297482 EP B1
2297518 EP A2
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297518 EP B1
2297524 EP A1
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297620 EP A1
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297635 EP A1
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/claims
2297806 EP A1
404 Client Error: Not Found for url: http

In [5]:
with open('Patent_dictionary_abstracts.json', 'r') as json_file:
    patent_dict_epo = json.load(json_file)

In [6]:
# Turn dictionary into a dataframe
patents_epo = pd.DataFrame.from_records(list(patent_dict_epo.values()))

# Reset the index and rename the columns
patents_epo.columns = ['country', 'number', 'kind', 'date', 'title_de', 'title_en', 'title_fr',
                           'abstract_de', 'abstract_en', 'abstract_fr', 'applicant_names', 'applicant_countries',
                           'inventor_names', 'inventor_countries', 'ipc_codes', 'cpc_codes', 'family_id', 
                           'application_number', 'url', 'claims_de', 'claims_en', 'claims_fr',
                           'description_de', 'description_en', 'description_fr']

# Print the dataframe
print(patents_epo)



      country   number kind      date  \
0          EP  3221170   B1  20200108   
1          EP  3221577   A1  20170927   
2          EP  3221577   B1  20190626   
3          EP  3221579   A1  20170927   
4          EP  3221579   B1  20230329   
...       ...      ...  ...       ...   
32470      EP  2439404   A1  20120411   
32471      EP  2439406   A1  20120411   
32472      EP  2439625   A2  20120411   
32473      EP  2439625   B1  20180829   
32474      EP  2439661   A1  20120411   

                                                title_de  \
0                        ANTRIEBSSTRANG FÜR EIN FAHRZEUG   
1      PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE   
2      PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE   
3      VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...   
4      VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...   
...                                                  ...   
32470  QUADRATISCHE AKTIVKÖRPER-GEPRESSTE WINDERZEUGU...   
32471  ROTORDREHVORRICHTUNG

In [8]:
keys_list = list(patent_dict_epo.keys())
pd.DataFrame(keys_list).to_csv("Patent_keys.csv", index=False)

In [7]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Display the dataframe
display(patents_epo)


,country,number,kind,date,title_de,title_en,title_fr,abstract_de,abstract_en,abstract_fr,applicant_names,applicant_countries,inventor_names,inventor_countries,ipc_codes,cpc_codes,family_id,application_number,url,claims_de,claims_en,claims_fr,description_de,description_en,description_fr
0,EP,3221170,B1,20200108,ANTRIEBSSTRANG FÜR EIN FAHRZEUG,POWERTRAIN FOR A VEHICLE,GROUPE MOTOPROPULSEUR POUR UN VÉHICULE,None,None,None,[Alpraaz AB],[SE],"[CHRISTIAN, V. Koenigsegg]",[SE],"[B60K 6/ 28 A I, B60K 6/ ...","[B60K 1/00, B60K 1/00, B60K 1/02, B60K 1/02, B...",54557409,15797070,None,None,None,None,None,None,None
1,EP,3221577,A1,20170927,"PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE","PUMP, IN PARTICULAR HIGH-PRESSURE FUEL PUMP","POMPE, EN PARTICULIER POMPE À CARBURANT À HAUT...",None,None,None,[Robert Bosch GmbH],[DE],"[MEIER, Gerhard, , SCHETTER, Markus]","[DE, IN]","[F02M 59/ 10 A I, F02M 59/ ...","[F02M 59/06, F02M 59/102, F02M 59/102, F02M 59...",54364321,15787551,None,None,None,None,None,None,None
2,EP,3221577,B1,20190626,"PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE","PUMP, IN PARTICULAR HIGH-PRESSURE FUEL PUMP","POMPE, EN PARTICULIER POMPE À CARBURANT À HAUT...",None,None,None,[Robert Bosch GmbH],[DE],"[MEIER, Gerhard, , SCHETTER, Markus]","[DE, IN]","[F02M 59/ 10 A I, F02M 59/ ...","[F02M 59/06, F02M 59/102, F02M 59/102, F02M 59...",54364321,15787551,None,None,None,None,None,None,None
3,EP,3221579,A1,20170927,VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...,WIND TURBINE CONDITION MONITORING METHOD AND S...,PROCÉDÉ ET SYSTÈME DE CONTRÔLE D'ÉTAT DE TURBI...,None,None,None,"[ABB Schweiz AG, , Hitachi Energy Switzerland AG]",[CH],"[YU, Rongrong, , CHEN, Niya, , CHEN, Yao]","[CN, CN, CN]","[F03D 7/ 00 A I, G01M 13/ ...","[F03D 17/00, F03D 17/00, G06N 20/00, G06N 20/0...",56013046,14906626,None,None,None,None,None,None,None
4,EP,3221579,B1,20230329,VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...,WIND TURBINE CONDITION MONITORING METHOD AND S...,PROCÉDÉ ET SYSTÈME DE CONTRÔLE D'ÉTAT DE TURBI...,None,None,None,"[ABB Schweiz AG, , Hitachi Energy Switzerland AG]",[CH],"[YU, Rongrong, , CHEN, Niya, , CHEN, Yao]","[CN, CN, CN]","[F03D 7/ 00 A I, G01M 13/ ...","[F03D 17/00, F03D 17/00, G06N 20/00, G06N 20/0...",56013046,14906626,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32470,EP,2439404,A1,20120411,QUADRATISCHE AKTIVKÖRPER-GEPRESSTE WINDERZEUGU...,SQUARE ACTIVE-BODY COMPRESSED WIND GENERATING ...,GÉNÉRATEUR À VENT COMPRIMÉ ACTIF À ANGLE DROIT,None,\n,None,"[LEI, YUENING, , LEI, SHENGQING]","[CN, CN]","[LEI, YUENING, , LEI, SHENGQING]","[CN, CN]","[F03D 3/ 02 A I, F03D 3/ ...","[F03D 3/005, F03D 3/005, F03D 3/005, F03D 3/02...",43297278,10782892,None,None,[1. A compressed-air wind turbine generator sy...,None,None,"[Technical Field, The present invention relate...",None
32471,EP,2439406,A1,20120411,ROTORDREHVORRICHTUNG FÜR WINDTURBINENGENERATOR...,ROTOR TURNING DEVICE AND ROTOR TURNING METHOD ...,DISPOSITIF DE ROTATION DE ROTOR ET PROCÉDÉ DE ...,None,\n,None,"[MITSUBISHI HEAVY INDUSTRIES, LTD]",[JP],"[EGOSHI, HIDEYA, , WATANABE, KUNIKAZU]","[JP, JP]","[F03D 7/ 04 A I, F16H 55/ ...","[F03D 1/06, F03D 1/06, F03D 15/00, F03D 80/50,...",44352031,10702986,None,None,[1. A rotor turning device for rotating a rota...,None,None,"[BACKGROUND OF THE INVENTION, Field of the Inv...",None
32472,EP,2439625,A2,20120411,VERFAHREN ZUR BEREITSTELLUNG EINER BENUTZEROBE...,METHOD FOR PROVIDING USER INTERFACE FOR EACH U...,PROCÉDÉ POUR FOURNIR UNE INTERFACE UTILISATEUR...,None,\n,None,"[SAMSUNG ELECTRONICS CO., LTD, , Samsung Elect...",[KR],"[CHEON, KA-WON, , YANG, PIL-SEUNG, , YOON, YEO...","[KR, KR, KR, KR]","[G06F 1/ 32 A I, G06F 3/ 04...","[G06F 1/3231, G06F 1/3231, G06F 1/3231, G06F 3...",43507415,10783598,None,None,[1. A method for providing a user interface (U...,None,None,"[Technical Field, Systems and methods consiste...",None
32473,EP,2439625,B1

In [8]:
patents_epo.drop_duplicates(subset=['country', 'number', 'kind', 'title_de', 'title_en', 'title_fr'], inplace=True)

In [11]:
with open('Missing_patents.txt', 'r') as file:
    missing_patents1 = [line.strip() for line in file]
with open('Missing_patents_2.txt', 'r') as file:
    missing_patents2 = [line.strip() for line in file]
with open('Missing_patents_3.txt', 'r') as file:
    missing_patents3 = [line.strip() for line in file]

In [20]:
# How many patents are not available on EPO OPS?
missing_patents = missing_patents1 + missing_patents2 + missing_patents3
len(missing_patents)

6065

## Step 2: Download bulk google patents data from Google Cloud (filtered using SQL queries)

In [15]:
patents_google = []
patents_google = pd.read_csv("Google_Patents_GCLOUD1.csv")
for i in range(2, 17):
    file = pd.read_csv(f"Google_Patents_GCLOUD{i}.csv")
    patents_google = pd.concat([patents_google, file], ignore_index=True)

In [17]:
patents_google

,publication_number,publication_date,filing_date,inventor,country_code,assignee,ipc,cpc,title_localized,abstract_localized,assignee_harmonized
0,EP-3362688-A1,20180822,20161014,"[LIN, Hua Hslang,HSU, YAW YUAN]",EP,"[Intex Marketing Ltd.,Lin, Hua Hsiang,Yaw Yuan...","{\n ""ipc"": [{\n ""code"": ""F04D17/10"",\n ...","{\n ""cpc"": [{\n ""code"": ""F04D25/166"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Pompe...","{\n ""abstract_localized"": [{\n ""text"": ""L\...","{\n ""assignee_harmonized"": [{\n ""name"": ""I..."
1,EP-3475776-A1,20190501,20170626,"[XU, Panpan,REN, LIU,MEI, Honghui]",EP,[Robert Bosch GmbH],"{\n ""ipc"": [{\n ""code"": ""G05B23/02"",\n ...","{\n ""cpc"": [{\n ""code"": ""G05B19/41875"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Systè...","{\n ""abstract_localized"": [{\n ""text"": ""L\...","{\n ""assignee_harmonized"": [{\n ""name"": ""B..."
2,EP-3676346-A1,20200708,20180814,"[MUELLER, DANNY,KNOLL, CHRISTIAN,WEINBERGER, P...",EP,"[Technische Universität Wien,Akademie Der Bild...","{\n ""ipc"": [{\n ""code"": ""F28D20/00"",\n ...","{\n ""cpc"": [{\n ""code"": ""C09K5/16"",\n ""...","{\n ""title_localized"": [{\n ""text"": ""Procé...","{\n ""abstract_localized"": [{\n ""text"": ""Th...","{\n ""assignee_harmonized"": [{\n ""name"": ""U..."
3,EP-3400605-A1,20181114,20161216,"[MEID, WOLFGANG,SCHAAR, INGO]",EP,[Eaton Intelligent Power Limited],"{\n ""ipc"": [{\n ""code"": ""H02H1/06"",\n ""...","{\n ""cpc"": [{\n ""code"": ""H01H47/22"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Steue...","{\n ""abstract_localized"": [{\n ""text"": ""Th...","{\n ""assignee_harmonized"": [{\n ""name"": ""E..."
4,EP-3453231-A1,20190313,20170505,"[GUTH, CHRISTIAN]",EP,[ZKW Group GmbH],"{\n ""ipc"": [{\n ""code"": ""H05B44/00"",\n ...","{\n ""cpc"": [{\n ""code"": ""H05B45/48"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Kraft...","{\n ""abstract_localized"": [{\n ""text"": ""Th...","{\n ""assignee_harmonized"": [{\n ""name"": ""Z..."
...,...,...,...,...,...,...,...,...,...,...,...
32994,EP-3635640-A1,20200415,20180523,"[MEHR, EDWARD,ELLIS, TIM,NOONE, Jordan]",EP,"[Relativity Space, Inc.]","{\n ""ipc"": [{\n ""code"": ""B29C67/00"",\n ...","{\n ""cpc"": [{\n ""code"": ""G06N3/084"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Comma...","{\n ""abstract_localized"": [{\n ""text"": ""Di...","{\n ""assignee_harmonized"": [{\n ""name"": ""R..."
32995,EP-3625882-A1,20200325,20180919,"[FU, DIANBO,CHEN, DONG]",EP,"[Huawei Technologies Co., Ltd.]","{\n ""ipc"": [{\n ""code"": ""H03K17/12"",\n ...","{\n ""cpc"": [{\n ""code"": ""H03K2217/0036"",\n...","{\n ""title_localized"": [{\n ""text"": ""Hybri...","{\n ""abstract_localized"": [{\n ""text"": ""A ...","{\n ""assignee_harmonized"": [{\n ""name"": ""H..."
32996,EP-3583749-A1,20191225,20180209,"[HOLTMANN, MICHELLE LYNN]",EP,"[Microsoft Technology Licensing, LLC]","{\n ""ipc"": [{\n ""code"": ""H04L12/58"",\n ...","{\n ""cpc"": [{\n ""code"": ""G06F40/211"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Benut...","{\n ""abstract_localized"": [{\n ""text"": ""Re...","{\n ""assignee_harmonized"": [{\n ""name"": ""M..."
32997,EP-3612937-A1,20200226,20180406,"[CEDOLA, Kent, D.,WALL, Larry, Marvin,BOBROV, ...",EP,"[Microsoft Technology Licensing, LLC]","{\n ""ipc"": [{\n ""code"": ""G06F9/50"",\n ""...","{\n ""cpc"": [{\n ""code"": ""H04L67/02"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Dynam...","{\n ""abstract_localized"": [{\n ""text"": ""Op...","{\n ""assignee_harmonized"": [{\n ""name"": ""M..."


In [18]:
# Drop duplicates from the dataframe
patents_google.drop_duplicates(keep='first', inplace=True)

In [19]:
# How many EPO patents are not in the google patents data?
# Get the list of publication numbers from the patents_google dataframe
publication_numbers = patents_google['publication_number'].tolist()
publication_numbers = [number.replace("-", "") for number in publication_numbers]

# Find the keys in patent_dict_epo that are not in the publication_numbers list
missing_keys = [key for key in patent_dict_epo.keys() if key not in publication_numbers]

# Print number of missing keys
len(missing_keys)


4534

these may be available online, allowing me to retrieve assignee and filing date

In [126]:
# Print missing keys for google patents online query
combined_string = "".join(
    [f'{value} \n' for value in missing_keys]
)
print(combined_string)
print("\n")


EP3221577A1 
EP3221579A1 
EP3221581A1 
EP3221655A1 
EP3221666A1 
EP3221764A1 
EP3221954A1 
EP3222001A1 
EP3222480A1 
EP3222821A1 
EP3222847A1 
EP3222867A1 
EP3222869A1 
EP3223034A1 
EP3223092A1 
EP3223131A1 
EP3223148A1 
EP3223167A1 
EP3223384A1 
EP3223587A2 
EP3223587A3 
EP3224075A1 
EP3224080A1 
EP3224126A1 
EP3224133A2 
EP3224474A1 
EP3224715A1 
EP3224821A1 
EP3224833A1 
EP3224890A1 
EP3224928A1 
EP3224937A1 
EP3224939A1 
EP3224940A1 
EP3225445A1 
EP3225446A1 
EP3225456A1 
EP3225489A1 
EP3225730A1 
EP3226090A1 
EP3226094A1 
EP3226295A1 
EP3226377A1 
EP3226486A1 
EP3226654A2 
EP3226654A3 
EP3226969A2 
EP3226970A2 
EP3227004A1 
EP3227018A1 
EP3227241A1 
EP3227533A1 
EP3227540A1 
EP3228044A1 
EP3228490A1 
EP3228546A1 
EP3228698A1 
EP3228863A1 
EP3229187A1 
EP3229343A1 
EP3229358A1 
EP3229358B1 
EP3229368A1 
EP3230104A1 
EP3230107A1 
EP3230132A2 
EP3230732A1 
EP3230866A1 
EP3231060A2 
EP3231679A1 
EP3232002A1 
EP3232048A1 
EP3232048B1 
EP3232053A1 
EP3232129A1 
EP3232392A1 
EP3232497A1 

In [22]:
# create new columns for cleaned data
patents_google['abstract_en'] = None
patents_google['title_en'] = None
patents_google['abstract_fr'] = None
patents_google['title_fr'] = None
patents_google['abstract_de'] = None
patents_google['title_de'] = None
patents_google['assignee_name'] = None
patents_google['assignee_country'] = None
patents_google['cpc_codes'] = None
patents_google['ipc_codes'] = None
patents_google.reset_index(drop=True, inplace=True)

In [ ]:
# parse the encoded data
for i in range(0, len(patents_google)):
    if pd.notnull(patents_google["abstract_localized"][i]): 
        abstract = json.loads(patents_google["abstract_localized"][i])
    if pd.notnull(patents_google["title_localized"][i]): 
        title = json.loads(patents_google["title_localized"][i])
    if pd.notnull(patents_google["assignee_harmonized"][i]): 
        assignee = json.loads(patents_google["assignee_harmonized"][i])
    if pd.notnull(patents_google["cpc"][i]): 
        cpc = json.loads(patents_google["cpc"][i])
    if pd.notnull(patents_google["ipc"][i]): 
        ipc = json.loads(patents_google["ipc"][i])

    # parse abstract
    try:
        for item in abstract['abstract_localized']: 
            text = html.unescape(item['text'])  # Decode HTML entities
            language = item['language']
        
            if language == 'en':
                if pd.isnull(patents_google.loc[i, 'abstract_en']):
                    patents_google.loc[i, 'abstract_en'] = text
                else:
                    patents_google.loc[i, 'abstract_en'] += ' ' + text
            elif language == 'fr':
                if pd.isnull(patents_google.loc[i, 'abstract_fr']):
                    patents_google.loc[i, 'abstract_fr'] = text
                else:
                    patents_google.loc[i, 'abstract_fr'] += ' ' + text
            elif language == 'de':
                if pd.isnull(patents_google.loc[i, 'abstract_de']):
                    patents_google.loc[i, 'abstract_de'] = text
                else:
                    patents_google.loc[i, 'abstract_de'] += ' ' + text
    except Exception as e:
        print(e)
        print(patents_google['publication_number'][i])
    
    # parse title
    for item in title['title_localized']:
        text = html.unescape(item['text'])  # Decode HTML entities
        language = item['language']
    
        if language == 'en':
            patents_google.loc[i, 'title_en'] = text
        elif language == 'fr':
            patents_google.loc[i, 'title_fr'] = text
        elif language == 'de':
            patents_google.loc[i, 'title_de'] = text
    
    # parse assignee
    if 'assignee_harmonized' in assignee and len(assignee['assignee_harmonized']) > 0:
        patents_google.loc[i, 'assignee_name'] = assignee['assignee_harmonized'][0]['name']
        patents_google.loc[i, 'assignee_country'] = assignee['assignee_harmonized'][0]['country_code']

    # parse cpc
    patents_google['cpc_codes'][i] = [item['code'] for item in cpc['cpc']]

    # parse ipc
    patents_google['ipc_codes'][i] = [item['code'] for item in ipc['ipc']]


In [24]:
patents_google

,publication_number,publication_date,filing_date,inventor,country_code,assignee,ipc,cpc,title_localized,abstract_localized,assignee_harmonized,abstract_en,title_en,abstract_fr,title_fr,abstract_de,title_de,assignee_name,assignee_country,cpc_codes,ipc_codes
0,EP-3362688-A1,20180822,20161014,"[LIN, Hua Hslang,HSU, YAW YUAN]",EP,"[Intex Marketing Ltd.,Lin, Hua Hsiang,Yaw Yuan...","{\n ""ipc"": [{\n ""code"": ""F04D17/10"",\n ...","{\n ""cpc"": [{\n ""code"": ""F04D25/166"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Pompe...","{\n ""abstract_localized"": [{\n ""text"": ""L\...","{\n ""assignee_harmonized"": [{\n ""name"": ""I...",A multifunctional air pump is disclosed includ...,Multifunctional air pump,L'invention concerne une pompe à air multifonc...,Pompe à air multifonctionnelle,None,Multifunktionale luftpumpe,INTEX MARKETING LTD,VG,"[F04D25/166, Y02B30/70, F04D29/503, F04D27/001...","[F04D17/10, F04D25/08, F04D27/00]"
1,EP-3475776-A1,20190501,20170626,"[XU, Panpan,REN, LIU,MEI, Honghui]",EP,[Robert Bosch GmbH],"{\n ""ipc"": [{\n ""code"": ""G05B23/02"",\n ...","{\n ""cpc"": [{\n ""code"": ""G05B19/41875"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Systè...","{\n ""abstract_localized"": [{\n ""text"": ""L\...","{\n ""assignee_harmonized"": [{\n ""name"": ""B...",A visual diagnostics (VIDX) system comprises a...,Visual diagnostics / analytics system and meth...,L'invention concerne un système de diagnostic ...,Système et procédé de diagnostic visuel/d'anal...,None,Visuelles diagnose-/analysesystem und verfahre...,BOSCH GMBH ROBERT,DE,"[G05B19/41875, G06F3/14, G05B2219/32196, G05B2...",[G05B23/02]
2,EP-3676346-A1,20200708,20180814,"[MUELLER, DANNY,KNOLL, CHRISTIAN,WEINBERGER, P...",EP,"[Technische Universität Wien,Akademie Der Bild...","{\n ""ipc"": [{\n ""code"": ""F28D20/00"",\n ...","{\n ""cpc"": [{\n ""code"": ""C09K5/16"",\n ""...","{\n ""title_localized"": [{\n ""text"": ""Procé...","{\n ""abstract_localized"": [{\n ""text"": ""Th...","{\n ""assignee_harmonized"": [{\n ""name"": ""U...",The invention relates to a method for thermo-c...,Method for thermo-chemical energy storage,L'invention concerne un procédé de stockage d'...,Procédé de stockage d'énergie thermochimique,Die Erfindung betrifft ein Verfahren zur therm...,Verfahren zur thermochemischen energiespeicherung,UNIV WIEN TECH,AT,"[C09K5/16, C09K5/08, Y02E60/14, Y02E60/14, F28...","[F28D20/00, C09K5/16]"
3,EP-3400605-A1,20181114,20161216,"[MEID, WOLFGANG,SCHAAR, INGO]",EP,[Eaton Intelligent Power Limited],"{\n ""ipc"": [{\n ""code"": ""H02H1/06"",\n ""...","{\n ""cpc"": [{\n ""code"": ""H01H47/22"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Steue...","{\n ""abstract_localized"": [{\n ""text"": ""Th...","{\n ""assignee_harmonized"": [{\n ""name"": ""E...",The invention relates to a control device for ...,Control device for an electromagnetic drive of...,L'invention concerne un dispositif de commande...,Dispositif de commande pour un entraînement él...,Die Erfindung betrifft eine Steuervorrichtung ...,Steuervorrichtung für einen elektromagnetische...,EATON INTELLIGENT POWER LTD,IE,"[H01H47/22, H01H47/22, H02H1/06, H02H1/06]","[H02H1/06, F02D41/20, H01H47/22]"
4,EP-3453231-A1,20190313,20170505,"[GUTH, CHRISTIAN]",EP,[ZKW Group GmbH],"{\n ""ipc"": [{\n ""code"": ""H05B44/00"",\n ...","{\n ""cpc"": [{\n ""code"": ""H05B45/48"",\n ...","{\n ""title_localized"": [{\n ""text"": ""Kraft...","{\n ""abstract_localized"": [{\n ""text"": ""Th...","{\n ""assignee_harmonized"": [{\n ""name"": ""Z...",The invention relates to a motor vehicle illum...,Motor vehicle illumination device,L'invention concerne un moyen d'éclairage de v...,Moyen d'éclairage de véhicule automobile,Kraftfahrzeug-Beleuchtungseinrichtung mit zumi...,Kraftfahrzeug-beleuchtungseinrichtung,ZKW GROUP GMBH,AT,"[H05B45/48, B60Q1/04, H02M7/06, H05B45/48, H02...",[H05B44/00]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27936,EP-3635640-A1,20200415,2018

In [25]:
patents_google.to_csv("Google_patents_cleaned.csv", index=False)

## Step 3: Query google patents directly with publications that are included in the patent_dict_epo but are missing in the google patents bulk dataset, to get assignees and filing dates

In [72]:
patents_google_online_raw = pd.read_csv("Google_patents_online.csv")

In [99]:
# Filter for those publications that are included in the patent_dict_epo
# the online advanced google patents search retrieved many more patents than publication numbers were provided in the search query
patents_google_online_filtered = patents_google_online_raw[patents_google_online_raw['id'].apply(lambda x: str(x.replace("-", "")) in patent_dict_epo.keys())]
len(patents_google_online_filtered)

5452

In [100]:
column_names = ['publication_number_GPO', 'title_GPO', 'assignee_name_GPO', 'inventor_name_GPO', 'priority_date_GPO', 'filing_date_GPO', 'publication_date_GPO', 'grant_date_GPO', 'result_link', 'representative_figure_link']
patents_google_online_filtered.columns = column_names
patents_google_online_slim = patents_google_online_filtered.drop(columns=['result_link', 'representative_figure_link'])
patents_google_online = patents_google_online_slim.drop_duplicates(subset=['publication_number_GPO', 'title_GPO'] , keep='first')
patents_google_online.reset_index(drop=True, inplace=True)

In [101]:
patents_google_online

,publication_number_GPO,title_GPO,assignee_name_GPO,inventor_name_GPO,priority_date_GPO,filing_date_GPO,publication_date_GPO,grant_date_GPO
0,EP-3224133-B1,Hybridantriebssystem,Graeme Eric Hawksley,Graeme Eric Hawksley,2014-11-25,2015-11-24,2020-07-22,2020-07-22
1,EP-3221170-B1,Powertrain for a vehicle,Alpraaz AB,V. Koenigsegg CHRISTIAN,2014-11-17,2015-11-17,2020-01-08,2020-01-08
2,EP-3224080-B1,Energy harvesting device for a transport vehicle,"Universidade do Porto, Addvolt S.A.","Adriano DA SILVA CARVALHO, Bruno Filipe AZEVED...",2014-11-25,2015-11-25,2018-12-19,2018-12-19
3,EP-3222480-B1,Fahrzeug und steuerungsverfahren für das fahrz...,Toyota Jidosha Kabushiki Kaisha,Kazunari Nagai,2016-03-24,2017-03-21,2019-01-02,2019-01-02
4,EP-3225445-A1,Fahrtantriebsvorrichtung für fahrzeug,Mitsubishi Jidosha Kogyo Kabushiki Kaisha,"Ryo Shimizu, Norihiko IKOMA, Makoto Kamachi, H...",2016-03-30,2017-03-29,2017-10-04,NaN
...,...,...,...,...,...,...,...,...
5323,EP-3231060-B1,A regulator rectifier device and a method for ...,Flash Electronics (India) Private Limited,Ramit VERMA,2014-12-11,2015-10-27,2018-11-21,2018-11-21
5324,EP-3233571-B1,Zweistufige indikatordimmerschaltung gesteuert...,Fca Us Llc,"Michael Mashkevich, David A. Lescamela, James ...",2014-12-15,2015-12-14,2018-11-21,2018-11-21
5325,EP-3240168-B1,Power supply and drive device with power facto...,Phase Motion Control S.p.A.,"Marco Venturini, Marco Calvini",2016-04-26,2017-04-26,2022-06-22,2022-06-22
5326,EP-3228698-B1,Hocheffiziente ethanolfermentierungsbakterien,"Honda Motor Co., Ltd.","Yoshiki Tsuchida, Ikumi KURIHARA, Tomohiro Ima...",2014-12-05,2014-12-05,2019-08-14,2019-08-14


In [102]:
patents_google_online.to_csv("Google_patents_online_filtered.csv", index=False)

## Combine datasources

In [78]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Display the dataframe
display(patents_epo)

,country,number,kind,publication_date,title_de,title_en,title_fr,abstract_de,abstract_en,abstract_fr,applicant_names,applicant_countries,inventor_names,inventor_countries,ipc_codes,cpc_codes,family_id,application_number,url,claims_de,claims_en,claims_fr,description_de,description_en,description_fr,publication_number_EPO
0,EP,3221170,B1,20200108,ANTRIEBSSTRANG FÜR EIN FAHRZEUG,POWERTRAIN FOR A VEHICLE,GROUPE MOTOPROPULSEUR POUR UN VÉHICULE,None,None,None,[Alpraaz AB],[SE],"[CHRISTIAN, V. Koenigsegg]",[SE],"[B60K 6/ 28 A I, B60K 6/ ...","[B60K 1/00, B60K 1/00, B60K 1/02, B60K 1/02, B...",54557409,15797070,None,None,None,None,None,None,None,EP-3221170-B1
1,EP,3221577,A1,20170927,"PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE","PUMP, IN PARTICULAR HIGH-PRESSURE FUEL PUMP","POMPE, EN PARTICULIER POMPE À CARBURANT À HAUT...",None,None,None,[Robert Bosch GmbH],[DE],"[MEIER, Gerhard, , SCHETTER, Markus]","[DE, IN]","[F02M 59/ 10 A I, F02M 59/ ...","[F02M 59/06, F02M 59/102, F02M 59/102, F02M 59...",54364321,15787551,None,None,None,None,None,None,None,EP-3221577-A1
2,EP,3221577,B1,20190626,"PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE","PUMP, IN PARTICULAR HIGH-PRESSURE FUEL PUMP","POMPE, EN PARTICULIER POMPE À CARBURANT À HAUT...",None,None,None,[Robert Bosch GmbH],[DE],"[MEIER, Gerhard, , SCHETTER, Markus]","[DE, IN]","[F02M 59/ 10 A I, F02M 59/ ...","[F02M 59/06, F02M 59/102, F02M 59/102, F02M 59...",54364321,15787551,None,None,None,None,None,None,None,EP-3221577-B1
3,EP,3221579,A1,20170927,VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...,WIND TURBINE CONDITION MONITORING METHOD AND S...,PROCÉDÉ ET SYSTÈME DE CONTRÔLE D'ÉTAT DE TURBI...,None,None,None,"[ABB Schweiz AG, , Hitachi Energy Switzerland AG]",[CH],"[YU, Rongrong, , CHEN, Niya, , CHEN, Yao]","[CN, CN, CN]","[F03D 7/ 00 A I, G01M 13/ ...","[F03D 17/00, F03D 17/00, G06N 20/00, G06N 20/0...",56013046,14906626,None,None,None,None,None,None,None,EP-3221579-A1
4,EP,3221579,B1,20230329,VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...,WIND TURBINE CONDITION MONITORING METHOD AND S...,PROCÉDÉ ET SYSTÈME DE CONTRÔLE D'ÉTAT DE TURBI...,None,None,None,"[ABB Schweiz AG, , Hitachi Energy Switzerland AG]",[CH],"[YU, Rongrong, , CHEN, Niya, , CHEN, Yao]","[CN, CN, CN]","[F03D 7/ 00 A I, G01M 13/ ...","[F03D 17/00, F03D 17/00, G06N 20/00, G06N 20/0...",56013046,14906626,None,None,None,None,None,None,None,EP-3221579-B1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32470,EP,2439404,A1,20120411,QUADRATISCHE AKTIVKÖRPER-GEPRESSTE WINDERZEUGU...,SQUARE ACTIVE-BODY COMPRESSED WIND GENERATING ...,GÉNÉRATEUR À VENT COMPRIMÉ ACTIF À ANGLE DROIT,None,\n,None,"[LEI, YUENING, , LEI, SHENGQING]","[CN, CN]","[LEI, YUENING, , LEI, SHENGQING]","[CN, CN]","[F03D 3/ 02 A I, F03D 3/ ...","[F03D 3/005, F03D 3/005, F03D 3/005, F03D 3/02...",43297278,10782892,None,None,[1. A compressed-air wind turbine generator sy...,None,None,"[Technical Field, The present invention relate...",None,EP-2439404-A1
32471,EP,2439406,A1,20120411,ROTORDREHVORRICHTUNG FÜR WINDTURBINENGENERATOR...,ROTOR TURNING DEVICE AND ROTOR TURNING METHOD ...,DISPOSITIF DE ROTATION DE ROTOR ET PROCÉDÉ DE ...,None,\n,None,"[MITSUBISHI HEAVY INDUSTRIES, LTD]",[JP],"[EGOSHI, HIDEYA, , WATANABE, KUNIKAZU]","[JP, JP]","[F03D 7/ 04 A I, F16H 55/ ...","[F03D 1/06, F03D 1/06, F03D 15/00, F03D 80/50,...",44352031,10702986,None,None,[1. A rotor turning device for rotating a rota...,None,None,"[BACKGROUND OF THE INVENTION, Field of the Inv...",None,EP-2439406-A1
32472,EP,2439625,A2,20120411,VERFAHREN ZUR BEREITSTELLUNG EINER BENUTZEROBE...,METHOD FOR PROVIDING USER INTERFACE FOR EACH U...,PROCÉDÉ POUR FOURNIR UNE INTERFACE UTILISATEUR...,None,\n,None,"[SAMSUNG ELECTRONICS CO., LTD, , Samsung Elect...",[KR],"[CHEON, KA-WON, , YANG, PIL-SEUNG, , YOON, YEO...","[KR, KR, KR, KR]","[G06F 1/ 32 A I, G06F 3/ 04...","[G06F 1/3231, G06F 1/3231, G06F 1/3231, G06F 3...",43507415,10783598,None,None

In [38]:
patents_epo.rename(columns={'date': 'publication_date'}, inplace=True)
patents_epo["publication_number_EPO"] = patents_epo["country"] + "-" + patents_epo["number"] + "-" + patents_epo["kind"]

In [103]:
merged_df = pd.merge(patents_epo, patents_google, left_on='publication_number_EPO', right_on='publication_number', how='left')


In [104]:
merged_df.columns

Index(['country', 'number', 'kind', 'publication_date_x', 'title_de_x',
       'title_en_x', 'title_fr_x', 'abstract_de_x', 'abstract_en_x',
       'abstract_fr_x', 'applicant_names', 'applicant_countries',
       'inventor_names', 'inventor_countries', 'ipc_codes_x', 'cpc_codes_x',
       'family_id', 'application_number', 'url', 'claims_de', 'claims_en',
       'claims_fr', 'description_de', 'description_en', 'description_fr',
       'publication_number_EPO', 'publication_number', 'publication_date_y',
       'filing_date', 'inventor', 'country_code', 'assignee', 'ipc', 'cpc',
       'title_localized', 'abstract_localized', 'assignee_harmonized',
       'abstract_en_y', 'title_en_y', 'abstract_fr_y', 'title_fr_y',
       'abstract_de_y', 'title_de_y', 'assignee_name', 'assignee_country',
       'cpc_codes_y', 'ipc_codes_y'],
      dtype='object')

In [105]:
# Select and rename columns
selected_columns = {'publication_date_x': 'publication_date_EPO', 
    'title_de_x': 'title_de_EPO',
    'title_en_x': 'title_en_EPO', 
    'title_fr_x': 'title_fr_EPO', 
    'abstract_de_x': 'abstract_de_EPO', 
    'abstract_en_x': 'abstract_en_EPO',
    'abstract_fr_x': 'abstract_fr_EPO', 
    'ipc_codes_x': 'ipc_codes_EPO', 
    'cpc_codes_x': 'cpc_codes_EPO',
    'inventor_names': 'inventor_EPO',
    'publication_date_y': 'publication_date_google', 
    'inventor' : 'inventor_google',  
    'abstract_en_y': 'abstract_en_google', 
    'title_en_y': 'title_en_google', 
    'abstract_fr_y': 'abstract_fr_google', 
    'title_fr_y': 'title_fr_google',
    'abstract_de_y': 'abstract_de_google', 
    'title_de_y': 'title_de_google', 
    'cpc_codes_y': 'cpc_codes_google', 
    'ipc_codes_y' : 'ipc_codes_google'}

# Select and rename columns in the merged DataFrame
patent_df_full = merged_df.rename(columns=selected_columns)

# Drop selected columns
patent_df_full.drop(columns=['url', 'publication_number' ,'country_code', 'assignee_harmonized', 'assignee', 'ipc', 'cpc', 'title_localized', 'abstract_localized'], inplace=True)

In [107]:
patent_df_complete = pd.merge(patent_df_full, patents_google_online, left_on='publication_number_EPO', right_on='publication_number_GPO', how='left')


In [ ]:
# show full dataframe
pd.set_option('display.max_columns', None)
patent_df_complete
# save dataframe as csv
# patent_df_complete.to_csv('patent_df_complete.csv', index=False)

,country,number,kind,publication_date_EPO,title_de_EPO,title_en_EPO,title_fr_EPO,abstract_de_EPO,abstract_en_EPO,abstract_fr_EPO,applicant_names,applicant_countries,inventor_EPO,inventor_countries,ipc_codes_EPO,cpc_codes_EPO,family_id,application_number,claims_de,claims_en,claims_fr,description_de,description_en,description_fr,publication_number_EPO,publication_date_google,filing_date,inventor_google,abstract_en_google,title_en_google,abstract_fr_google,title_fr_google,abstract_de_google,title_de_google,assignee_name,assignee_country,cpc_codes_google,ipc_codes_google,publication_number_GPO,title_GPO,assignee_name_GPO,inventor_name_GPO,priority_date_GPO,filing_date_GPO,publication_date_GPO,grant_date_GPO
0,EP,3221170,B1,20200108,ANTRIEBSSTRANG FÜR EIN FAHRZEUG,POWERTRAIN FOR A VEHICLE,GROUPE MOTOPROPULSEUR POUR UN VÉHICULE,None,None,None,[Alpraaz AB],[SE],"[CHRISTIAN, V. Koenigsegg]",[SE],"[B60K 6/ 28 A I, B60K 6/ ...","[B60K 1/00, B60K 1/00, B60K 1/02, B60K 1/02, B...",54557409,15797070,None,None,None,None,None,None,EP-3221170-B1,20200108.0,20151117.0,"[CHRISTIAN, V. Koenigsegg]",None,Powertrain for a vehicle,None,Groupe motopropulseur pour un véhicule,None,Antriebsstrang für ein fahrzeug,ALPRAAZ AB,SE,"[Y10S903/946, B60K6/28, B60K6/50, B60K2007/006...","[B60K6/28, B60K17/356, B60K6/38, B60W20/00, B6...",EP-3221170-B1,Powertrain for a vehicle,Alpraaz AB,V. Koenigsegg CHRISTIAN,2014-11-17,2015-11-17,2020-01-08,2020-01-08
1,EP,3221577,A1,20170927,"PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE","PUMP, IN PARTICULAR HIGH-PRESSURE FUEL PUMP","POMPE, EN PARTICULIER POMPE À CARBURANT À HAUT...",None,None,None,[Robert Bosch GmbH],[DE],"[MEIER, Gerhard, , SCHETTER, Markus]","[DE, IN]","[F02M 59/ 10 A I, F02M 59/ ...","[F02M 59/06, F02M 59/102, F02M 59/102, F02M 59...",54364321,15787551,None,None,None,None,None,None,EP-3221577-A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EP-3221577-A1,"Pumpe, insbesondere eine kraftstoffhochdruckpu...",Robert Bosch GmbH,"Gerhard Meier, Markus Schetter",2014-11-21,2015-10-27,2017-09-27,NaN
2,EP,3221577,B1,20190626,"PUMPE, INSBESONDERE EINE KRAFTSTOFFHOCHDRUCKPUMPE","PUMP, IN PARTICULAR HIGH-PRESSURE FUEL PUMP","POMPE, EN PARTICULIER POMPE À CARBURANT À HAUT...",None,None,None,[Robert Bosch GmbH],[DE],"[MEIER, Gerhard, , SCHETTER, Markus]","[DE, IN]","[F02M 59/ 10 A I, F02M 59/ ...","[F02M 59/06, F02M 59/102, F02M 59/102, F02M 59...",54364321,15787551,None,None,None,None,None,None,EP-3221577-B1,20190626.0,20151027.0,"[MEIER, GERHARD,SCHETTER, MARKUS]",None,"Pump, in particular high-pressure fuel pump",None,"Pompe, en particulier pompe à carburant à haut...",None,"Pumpe, insbesondere eine kraftstoffhochdruckpumpe",BOSCH GMBH ROBERT,DE,"[F02M59/445, F05C2225/00, F16C2208/02, F02M220...","[F04B1/04, F16C33/20, F02M59/10, F02M59/44]",EP-3221577-B1,"Pump, in particular high-pressure fuel pump",Robert Bosch GmbH,"Gerhard Meier, Markus Schetter",2014-11-21,2015-10-27,2019-06-26,2019-06-26
3,EP,3221579,A1,20170927,VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...,WIND TURBINE CONDITION MONITORING METHOD AND S...,PROCÉDÉ ET SYSTÈME DE CONTRÔLE D'ÉTAT DE TURBI...,None,None,None,"[ABB Schweiz AG, , Hitachi Energy Switzerland AG]",[CH],"[YU, Rongrong, , CHEN, Niya, , CHEN, Yao]","[CN, CN, CN]","[F03D 7/ 00 A I, G01M 13/ ...","[F03D 17/00, F03D 17/00, G06N 20/00, G06N 20/0...",56013046,14906626,None,None,None,None,None,None,EP-3221579-A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EP-3221579-A1,Verfahren und system zur überwachung des zusta...,ABB Schweiz AG,"Rongrong Yu, Niya CHEN, Yao Chen",2014-11-18,2014-11-18,2017-09-27,NaN
4,EP,3221579,B1,20230329,VERFAHREN UND SYSTEM ZUR ÜBERWACHUNG DES ZUSTA...,WIND TURBINE CONDITION MONITORING METHOD AND S...,PROCÉDÉ ET SYSTÈME DE CONTRÔLE D'ÉTAT DE TURBI...,None,None,None,"[ABB Schweiz AG, , Hitachi Energy Switzerland AG]",[CH],"[YU, Rongrong, , CHEN, Niya, , CHEN, Yao]","[CN, CN, CN]","[F03D 7/ 00 A I, G01M 13/ ...","[F03D 17/00, F03D 17/00, G06N 20/00